## 문제 1

### 1-1. 분류 평가지표 중 '정밀도'와 '재현율'에 대한 개념과 이 둘은 어떤 관계인지 설명해주세요.

정밀도는 예측을 positiv로 한 건수 중 예측 결과와 실제 값이 동일한 건수를 말하고, 재현율은 실제값이 positive인 건수 중 예측 결과와 실제 값이 동일한 건수를 의미한다. Positive 데이터 세트의 예측 성능에 좀 더 초점을 맞춘 평가 지표이다. Positive 예측값을 결정하는 확률의 기준인 임계값의 변화에 따라 재현율과 정밀도가 변화한다. 

### 1-2.  '정밀도'와 '재현율'의 관계로 인하여 다른 어떤 평가지표를 쓰는 것이 좋은지 이유와 함께 설명해주세요.

불균형한 데이터 세트에서 정밀도와 제현율을 사용하는 것이 좋다. 불균형한 이진분류 데이터 세트에서는 Positive 데이터 건수가 작아 Negative로 예측 정확도가 높아진다. 다른 지표인 정확도 지표는 Negative에 대한 예측 정확도만으로 분류의 정확도가 매우 높게 나타날 수 있어, Positve 데이터 세트의 예측 성능에 초점을 맞춘 평가 지표인 정밀도와 제현율을 사용하면 좋다. 

***

## 문제 2

### 정밀도와 재현율이 각각 더 중요한 사례를 들어주세요.
- 정밀도가 중요한 사례와 재현율이 중요한 사례 총 2가지를 적어주셔야 합니다.
- 되도록이면 PPT 이외의 사례를 들어주세요!

정밀도가 중요한 사례로는 안 좋은 성분이 있는 negative인 화장품을 문제가 없는 positive한 화장품으로 분류할 경우를 들 수 있다. 

재현율이 중요한 사례로는 코로나 확진자인 positive인 사람을 negative로 판단했을 경우를 들 수 있다. 

***

## 문제3

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import Binarizer

In [3]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

### [ 종양종류 예측해보기 ]
- load_breast_cancer은 위스콘신 유방암 데이터로 악성종양(malignant)과 양성종양(benign)으로 구분되는 데이터세트 입니다.
- 타깃 레이블 값의 종류는 악성이 '0'으로 양성이 '1'로 되어 있습니다.

#### 기본 제공 코드

In [10]:
# 데이텃셋 로드 및 X,y 지정
dataset = load_breast_cancer()
X_features=dataset.data
y_label=dataset.target

In [5]:
# `train_test_split`으로 train, test 데이터 분리

X_train,X_test,y_train,y_test = train_test_split(X_features, y_label, test_size=0.3, random_state=11)
print(X_train.shape, X_test.shape)

(398, 30) (171, 30)


In [6]:
# RandomForest 객체 생성 후 예측 수행

from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state = 0)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
pred_proba = rf_clf.predict_proba(X_test)[:,1]

### 3-1. 정확도, 정밀도, 재현율, 오차행렬을 산출하세요.
- get_clf_eval 함수를 활용하세요.

In [12]:
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(accuracy, precision, recall))

get_clf_eval(y_test, pred)

오차 행렬
[[ 56   5]
 [  1 109]]
정확도: 0.9649, 정밀도: 0.9561, 재현율: 0.9909


### 3-2. 임계값이 0.3, 0.5, 0.7, 0.9일때 오차행렬, 정확도, 정밀도, 재현율을 구하세요.

In [13]:
thresholds = [ 0.3, 0.5, 0.7, 0.9]

In [16]:
def get_eval_by_threshold(y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold = custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print('임계값 : ', custom_threshold)
        get_clf_eval(y_test, custom_predict)
        print('')
        
get_eval_by_threshold(y_test, pred_proba.reshape(-1,1), thresholds)

임계값 :  0.3
오차 행렬
[[ 51  10]
 [  0 110]]
정확도: 0.9415, 정밀도: 0.9167, 재현율: 1.0000

임계값 :  0.5
오차 행렬
[[ 56   5]
 [  1 109]]
정확도: 0.9649, 정밀도: 0.9561, 재현율: 0.9909

임계값 :  0.7
오차 행렬
[[ 59   2]
 [  4 106]]
정확도: 0.9649, 정밀도: 0.9815, 재현율: 0.9636

임계값 :  0.9
오차 행렬
[[59  2]
 [11 99]]
정확도: 0.9240, 정밀도: 0.9802, 재현율: 0.9000



### 3-3. 위의 임곗값들 중 어떤 것이 가장 적절할까요? 답과 그 이유를 적어주세요.

악성종양(Negative)인데 양성종양(Positive)으로 판별하는 경우를 줄여야한다. 따라서 FP가 영향을 미치는 정밀도가 높은 것을 선택해야한다. 
정밀도가 높은 임계값이 0.9가 적절하다고 생각한다. 

이 문제의 답은 분석가들의 insight의 차이로 결정되기 때문에 정확한 정답은 없다. 
정밀도와 재현율의 Trade-off 성질로 인해 사용하는 F1 score에 대해 생각해본다면, 정확도도 높고 정밀도와 재현율의 차이가 크지 않은 임계값 0.7이 가장 적당하다고 할 수 있다. 

***

## 문제 4

### 4-1. F1 스코어의 개념과 어떨 때 상대적으로 높은 값를 가지는지 적어주세요.

정밀도와 재현율의 조화평균으로 어느 한쪽으로 치우쳐 지지 않을 때 높은 값을 가진다. 

### 4-2. F1 스코어를 구하세요.

In [17]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test, pred)
print(f'f1 score : {f1:.3f}')

f1 score : 0.973


### 4-3. `get_clf_eval()` 함수에 F1 스코어를 구하는 코드를 추가하세요.

In [18]:
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3: .4f}'.format(accuracy, precision, recall, f1))

### 4-4. <U>F1 스코어 >= 0.98</U>이 되도록 값을 출력하세요.  (다른 평가 지표의 값은 무시)
  
`Hint : 임계값 조정`

In [24]:
custom_threshold = 0.6
pred_proba_1 = pred_proba.reshape(-1,1)
binarizer = Binarizer(threshold = custom_threshold).fit(pred_proba_1)
custom_predict = binarizer.transform(pred_proba_1)

get_clf_eval(y_test, custom_predict)

오차 행렬
[[ 59   2]
 [  2 108]]
정확도: 0.9766, 정밀도: 0.9818, 재현율: 0.9818, F1: 0.9818


***

## 문제 5

### 5-1. ROC-AUC가 무엇인지 설명하세요. 

In [14]:
# ROC 곡선 : Roc곡선은 거짓 양성 비율에 대한 진짜 양성 비율의 곡선이다. 
# TPR(True Positive Rate) : 실제 양성을 제대로 양성이라고 예측한 것이다. 
# TNR(True Negative Rate) : 실제 음성을 제대로 음성이라고 예측한 것이다. 
# AUC(Area Under Curve) : Roc곡선의 면적의 값이다. 

### 5-2. ROC-AUC 값을 구하세요. 

In [26]:
from sklearn.metrics import roc_auc_score

fr_roc_score = roc_auc_score(y_test,pred_proba)
print('fr_ROC AUC 값: {0:.4f}'.format(fr_roc_score))

fr_ROC AUC 값: 0.9903
